# 04_eval – Eval-Abschlusslauf: Finale Konfiguration für die Einreichung

## Kontext

Dieses Notebook ist der **finale Eval-Durchlauf** – die Generierung der
Einreichungs-VTTs auf dem ungesehenen Eval-Set mit der in `02k_` gefundenen
optimalen Konfiguration (identisch zu `04_dev_final_results`, nur `SPLIT = "EVAL"`).

## Technische Besonderheit: OOM-Abbrüche

Das Eval-Set ist größer als das Dev-Set. Beim ersten Durchlauf trat bei
Session 129 ein Out-of-Memory-Fehler auf. Die fehlenden Sessions wurden in
**drei weiteren Durchläufen** nachgeholt:

| Durchlauf | Zellen | Fehlende Sessions | Grund |
|-----------|--------|-------------------|-------|
| 1 (Zelle 10) | Alle Sessions | Abbruch bei session_129 | OOM |
| 2 (Zellen 11–12) | 38 fehlende Sessions | session_148, session_80 fehlgeschlagen | OOM |
| 3 (Zellen 13–14) | session_148, session_80 | session_148 fehlgeschlagen | OOM |
| 4 (Zellen 15–19) | session_148 | `PYTORCH_CUDA_ALLOC_CONF=expandable_segments` | Erfolg |

Der vierte Durchlauf nutzt `expandable_segments=True`, was PyTorch erlaubt,
CUDA-Speicher fragmentierungsresistenter zu verwalten.

## Konfiguration

Identisch zu `04_dev_final_results` (min_on=1.0, min_off=1.2, beam=12, len=20, BL4).
Keine WER-Evaluation: Das Eval-Set hat keine Labels → Ergebnisse werden erst nach Einreichung bekannt.

## 1 – GPU-Check

In [1]:
!nvidia-smi

Fri Feb  6 02:27:45 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   41C    P0             84W /  500W |   25069MiB /  81920MiB |      6%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [1]:
import os

# Physische GPU-Auswahl
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

## 2 – CUDA-Verifikation

In [9]:
import torch, gc, glob
from pathlib import Path
import pandas as pd

In [3]:
print("CUDA devices:", torch.cuda.device_count())
print("Device 0 name (sollte A100 sein):", torch.cuda.get_device_name(0))

CUDA devices: 1
Device 0 name (sollte A100 sein): NVIDIA A100-SXM4-80GB


## 3 – Setup: Arbeitsverzeichnis & Imports

In [2]:
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

import sys
if project_baseline_path not in sys.path:
    sys.path.append(project_baseline_path)

from script.pg_utils_experiments import run_inference_for_experiment, append_eval_results_for_experiments

/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:22: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## 4 – Segmentierungs-Patch (identisch zu `04_dev_`)

In [3]:
from contextlib import contextmanager
import script.inference as inf  # das ist das Modul, das pg_utils_experiments nutzt

@contextmanager
def patch_avsr_segmentation(min_on=None, min_off=None):
    orig = inf.InferenceEngine.chunk_video

    if min_on is None and min_off is None:
        yield
        return

    def patched(self, video_path, asd_path=None, max_length=15):
        if asd_path is not None:
            with open(asd_path, "r") as f:
                asd = inf.json.load(f)

            frames = sorted([int(f) for f in asd.keys()])
            if not frames:
                return []
            min_frame = min(frames)

            params = {"max_chunk_size": max_length}
            if min_on is not None:
                params["min_duration_on"] = float(min_on)
            if min_off is not None:
                params["min_duration_off"] = float(min_off)

            segments_by_frames = inf.segment_by_asd(asd, params)
            return [((seg[0] - min_frame) / 25, (seg[-1] - min_frame) / 25) for seg in segments_by_frames]

        return orig(self, video_path, asd_path, max_length)

    inf.InferenceEngine.chunk_video = patched
    try:
        yield
    finally:
        inf.InferenceEngine.chunk_video = orig

def _tag(x: float) -> str:
    return str(x).replace(".", "p")

## 5 – Split-Konfiguration

`SPLIT = "EVAL"` → `data-bin/eval` (Unterschied zu `04_dev_`).

In [4]:
SPLIT = "EVAL"
DATA_ROOT = "data-bin/dev" if SPLIT == "DEV" else "data-bin/eval"  # ggf. anpassen

## 6 – Experiment-Definition

Identisch zu `04_dev_`, nur `SPLIT`-Präfix.

In [5]:
beam_size  = 12
max_length = 20
best_min_on  = 1.0
best_min_off = 1.2

exp_name = f"{SPLIT}_final_bugfix_mdOn{_tag(best_min_on)}_mdOff{_tag(best_min_off)}_bs{beam_size}_len{max_length}"
BASE_MODELS = {
    "avsr_cocktail_finetuned": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail_mcorec_finetune",
    },
}
EXPERIMENTS = {
    exp_name: {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": beam_size,
        "max_length": max_length,
        "comment": f"{SPLIT} FINAL: AVSR override min_on={best_min_on}, min_off={best_min_off}",
    }
}

## 7 – Session-Erkennung

In [13]:
# Sessions für den Split finden
session_dirs = sorted([p for p in glob.glob(f"{DATA_ROOT}/session_*")
                       if Path(p, "metadata.json").exists()])
session_ids = [Path(p).name for p in session_dirs]
print("Split:", SPLIT, "| Sessions:", len(session_ids))
print(session_ids)

Split: EVAL | Sessions: 67
['session_05', 'session_06', 'session_07', 'session_08', 'session_09', 'session_10', 'session_107', 'session_108', 'session_109', 'session_11', 'session_110', 'session_111', 'session_112', 'session_113', 'session_114', 'session_115', 'session_116', 'session_117', 'session_118', 'session_119', 'session_12', 'session_120', 'session_121', 'session_122', 'session_123', 'session_124', 'session_125', 'session_126', 'session_127', 'session_128', 'session_129', 'session_13', 'session_130', 'session_131', 'session_14', 'session_142', 'session_143', 'session_144', 'session_145', 'session_146', 'session_147', 'session_148', 'session_149', 'session_15', 'session_151', 'session_16', 'session_17', 'session_18', 'session_19', 'session_31', 'session_32', 'session_34', 'session_35', 'session_36', 'session_37', 'session_38', 'session_39', 'session_73', 'session_74', 'session_75', 'session_76', 'session_77', 'session_78', 'session_79', 'session_80', 'session_81', 'session_82']


## 8 – Inference: Erster Durchlauf (alle Sessions)

Abbruch bei `session_129` wegen OOM. Alle Sessions ab `session_122` aufwärts
müssen im zweiten Durchlauf nachgeholt werden.

In [10]:
# ======= Inference pro Session =======
for session_dir in session_dirs:
    print("\nRUN:", SPLIT, Path(session_dir).name)
    with patch_avsr_segmentation(min_on=best_min_on, min_off=best_min_off):
        run_inference_for_experiment(
            exp_name=exp_name,
            base_models=BASE_MODELS,
            experiments=EXPERIMENTS,
            session_dir=session_dir,
        )
    torch.cuda.empty_cache()
    gc.collect()


RUN: EVAL session_05

Starte Inference für Experiment: EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  session_dir     = data-bin/eval/session_05
  comment         = EVAL FINAL: AVSR override min_on=1.0, min_off=1.2
Loading avsr_cocktail model...


/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_05


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [01:02<05:11, 62.32s/it]

...


RUN: EVAL session_129

Starte Inference für Experiment: EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  session_dir     = data-bin/eval/session_129
  comment         = EVAL FINAL: AVSR override min_on=1.0, min_off=1.2
Loading avsr_cocktail model...
Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_129


Processing speakers:   0%|          | 0/4 [00:00<?, ?it/s]


Processing speaker spk_0 track 1 of 2: 100%|██████████| 5/5 [00:12<00:00,  2.46s/it]

Processing speakers:  25%|██▌       | 1/4 [01:45<05:16, 105.46s/it]


Processing speaker spk_1 track 1 of 2: 100%|██████████| 7/7 [00:05<00:00,  1.35it/s]

Processing speakers:  25%|██▌       | 1/4 [01:51<05:33, 111.30s/it]


RuntimeError: Could not open input file: data-bin/eval/session_129/speakers/spk_1/central_crops/track_01_lip.av.mp4 Invalid data found when processing input

## 9 – Resume-Liste Durchlauf 2

Nach OOM-Abbruch bei `session_129` werden alle Sessions ab diesem Punkt
explizit aufgelistet, insgesamt 38 Sessions.

In [11]:
# Erster Durchlauf ist bei Session 129 abgebrochen, deshalb erneuter Durchlauf
missing_session_ids = [
    "session_122",
    "session_129",
    "session_13",
    "session_130",
    "session_131",
    "session_14",
    "session_142",
    "session_143",
    "session_144",
    "session_145",
    "session_146",
    "session_147",
    "session_148",
    "session_149",
    "session_15",
    "session_151",
    "session_16",
    "session_17",
    "session_18",
    "session_19",
    "session_31",
    "session_32",
    "session_34",
    "session_35",
    "session_36",
    "session_37",
    "session_38",
    "session_39",
    "session_73",
    "session_74",
    "session_75",
    "session_76",
    "session_77",
    "session_78",
    "session_79",
    "session_80",
    "session_81",
    "session_82",
]


## 10 – Resume Durchlauf 2

`try/except/finally`-Wrapper pro Session: Bei OOM wird die Session zur `failed`-Liste
hinzugefügt und der Lauf weitergeführt, statt komplett abzubrechen.

In [12]:
from pathlib import Path

missing_session_dirs = [str(Path(DATA_ROOT) / sid) for sid in missing_session_ids]

failed = []
for session_dir in missing_session_dirs:
    sid = Path(session_dir).name
    print("\nRUN:", SPLIT, sid)
    try:
        with patch_avsr_segmentation(min_on=best_min_on, min_off=best_min_off):
            run_inference_for_experiment(
                exp_name=exp_name,
                base_models=BASE_MODELS,
                experiments=EXPERIMENTS,
                session_dir=session_dir,
            )
    except Exception as e:
        print("FAILED:", sid, "->", repr(e))
        failed.append(sid) # Session merken, nicht abbrechen
    finally:
        torch.cuda.empty_cache()
        gc.collect()

print("Failed:", failed) # Ergebnis: ['session_148', 'session_80']



RUN: EVAL session_122

Starte Inference für Experiment: EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  session_dir     = data-bin/eval/session_122
  comment         = EVAL FINAL: AVSR override min_on=1.0, min_off=1.2
Loading avsr_cocktail model...
Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_122


Processing speakers:   0%|          | 0/5 [00:00<?, ?it/s]


Processing speaker spk_0 track 1 of 2: 100%|██████████| 1/1 [00:00<00:00,  1.79it/s]

Processing speakers:  20%|██        | 1/5 [00:09<00:39,  9.84s/it]


Processing speakers:  40%|████      | 2/5 [01:42<02:56, 58.84s/it]


Processing speakers:  60%|██████    | 3/5 [02:26<01:44, 52.03s/it]

...


RUN: EVAL session_82

Starte Inference für Experiment: EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  session_dir     = data-bin/eval/session_82
  comment         = EVAL FINAL: AVSR override min_on=1.0, min_off=1.2
Loading avsr_cocktail model...
Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_82


Processing speakers:   0%|          | 0/2 [00:00<?, ?it/s]


Processing speaker spk_0 track 1 of 3: 0it [00:00, ?it/s]

Processing speaker spk_0 track 2 of 3: 100%|██████████| 23/23 [00:24<00:00,  1.08s/it]

Processing speakers:  50%|█████     | 1/2 [00:35<00:35, 35.70s/it]


Processing speaker spk_1 track 1 of 5: 100%|██████████| 2/2 [00:01<00:00,  1.75it/s]

Processing speaker spk_1 track 2 of 5: 100%|██████████| 8/8 [00:04<00:00,  1.75it/s]

Processing speaker spk_1 track 3 of 5: 100%|██████████| 2/2 [00:01<00:00,  1.99it/s]

Processing speaker spk_1 track 4 of 5: 100%|██████████| 7/7 [00:07<00:00,  1.01s/it]

Processing speakers: 100%|██████████| 2/2 [01:08<00:00, 34.46s/it]


Failed: ['session_148', 'session_80']


## 11 – Resume-Liste Durchlauf 3

Nur noch `session_148` und `session_80` fehlgeschlagen (OOM trotz try/except).

In [17]:
# Im zweiten Durchlauf schlugen Session 148 und 80 fehl, deshalb erneuter Durchlauf
missing_session_ids = [
    "session_148",
    "session_80",
]


## 12 – Resume Durchlauf 3 (identisch zu Durchlauf 2)

In [18]:
from pathlib import Path

missing_session_dirs = [str(Path(DATA_ROOT) / sid) for sid in missing_session_ids]

failed = []
for session_dir in missing_session_dirs:
    sid = Path(session_dir).name
    print("\nRUN:", SPLIT, sid)
    try:
        with patch_avsr_segmentation(min_on=best_min_on, min_off=best_min_off):
            run_inference_for_experiment(
                exp_name=exp_name,
                base_models=BASE_MODELS,
                experiments=EXPERIMENTS,
                session_dir=session_dir,
            )
    except Exception as e:
        print("FAILED:", sid, "->", repr(e))
        failed.append(sid)
    finally:
        torch.cuda.empty_cache()
        gc.collect()

print("Failed:", failed) # Ergebnis: ['session_148']



RUN: EVAL session_148

Starte Inference für Experiment: EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  session_dir     = data-bin/eval/session_148
  comment         = EVAL FINAL: AVSR override min_on=1.0, min_off=1.2
Loading avsr_cocktail model...
Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_148


Processing speakers:   0%|          | 0/8 [00:00<?, ?it/s]


Processing speakers:  12%|█▎        | 1/8 [00:44<05:08, 44.07s/it]


Processing speaker spk_1 track 1 of 5: 100%|██████████| 8/8 [00:06<00:00,  1.28it/s]

Processing speaker spk_1 track 2 of 5: 100%|██████████| 9/9 [00:17<00:00,  1.97s/it]

Processing speaker spk_1 track 3 of 5: 100%|██████████| 4/4 [00:13<00:00,  3.48s/it]

Processing speaker spk_1 track 4 of 5: 100%|██████████| 8/8 [00:23<00:00,  2.93s/it]

Processing speakers:  25%|██▌       | 2/8 [01:55<06:01, 60.20s/it]


Processing speaker spk_2 track 1 of 2: 100%|██████████| 28/28 [01:02<00:00,  2.25s/it]

Processing speakers:  38%|███▊      | 3/8 [03:07<05:27, 65.47s/it]

...


RUN: EVAL session_80

Starte Inference für Experiment: EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  session_dir     = data-bin/eval/session_80
  comment         = EVAL FINAL: AVSR override min_on=1.0, min_off=1.2
Loading avsr_cocktail model...
Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_80


Processing speakers:   0%|          | 0/2 [00:00<?, ?it/s]


Processing speaker spk_0 track 1 of 3: 100%|██████████| 7/7 [00:05<00:00,  1.32it/s]

Processing speaker spk_0 track 2 of 3: 100%|██████████| 8/8 [00:05<00:00,  1.38it/s]

Processing speakers:  50%|█████     | 1/2 [00:19<00:19, 19.18s/it]


Processing speaker spk_1 track 1 of 3: 100%|██████████| 18/18 [00:27<00:00,  1.52s/it]

Processing speaker spk_1 track 2 of 3: 100%|██████████| 1/1 [00:00<00:00,  2.14it/s]

Processing speakers: 100%|██████████| 2/2 [00:54<00:00, 27.09s/it]


Failed: ['session_148']


## 13 – CUDA-Konfiguration für Durchlauf 4

`PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True` erlaubt PyTorch,
CUDA-Speicher in kleineren, nicht-zusammenhängenden Blöcken zu verwalten.
Das reduziert Fragmentierung und ermöglicht die Verarbeitung von `session_148`,
die besonders viele/lange Video-Chunks enthält.

In [6]:
import os

# expandable_segments: PyTorch darf CUDA-Speicher fragmentierungsresistenter verwalten
# → ermöglicht Verarbeitung von session_148 die ohne dieses Flag OOM erzeugt

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import torch


In [7]:
import torch
print(torch.cuda.get_device_name(0))


NVIDIA A100-SXM4-80GB


In [8]:
from tqdm import tqdm

## 14 – Resume-Liste Durchlauf 4

Nur noch `session_148` übrig.

In [10]:
# Im dritten Durchlauf schlug Session 148 fehl, deshalb erneuter Durchlauf
missing_session_ids = [
    "session_148",
]


## 15 – Resume Durchlauf 4 (session_148 mit expandable_segments)

Mit `expandable_segments:True` kann PyTorch CUDA-Speicher fragmentierungsresistenter
verwalten. `session_148` wird so erfolgreich verarbeitet.

In [11]:
from pathlib import Path

missing_session_dirs = [str(Path(DATA_ROOT) / sid) for sid in missing_session_ids]

failed = []
for session_dir in missing_session_dirs:
    sid = Path(session_dir).name
    print("\nRUN:", SPLIT, sid)
    try:
        with patch_avsr_segmentation(min_on=best_min_on, min_off=best_min_off):
            run_inference_for_experiment(
                exp_name=exp_name,
                base_models=BASE_MODELS,
                experiments=EXPERIMENTS,
                session_dir=session_dir,
            )

    except Exception as e:
        print("FAILED:", sid, "->", repr(e))
        failed.append(sid)
    finally:
        torch.cuda.empty_cache()
        gc.collect()

print("Failed:", failed)



RUN: EVAL session_148

Starte Inference für Experiment: EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  session_dir     = data-bin/eval/session_148
  comment         = EVAL FINAL: AVSR override min_on=1.0, min_off=1.2
Loading avsr_cocktail model...


/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_148


Processing speakers:   0%|          | 0/8 [00:00<?, ?it/s]


Processing speakers:  12%|█▎        | 1/8 [00:30<03:36, 30.97s/it]

...


Processing speakers:  62%|██████▎   | 5/8 [04:28<02:49, 56.60s/it]


Processing speaker spk_5 track 1 of 2: 100%|██████████| 7/7 [00:19<00:00,  2.82s/it]

Processing speakers:  75%|███████▌  | 6/8 [05:31<01:57, 58.68s/it]


Processing speaker spk_6 track 1 of 2: 100%|██████████| 16/16 [00:14<00:00,  1.09it/s]

Processing speakers:  88%|████████▊ | 7/8 [05:50<00:45, 45.68s/it]


Processing speaker spk_7 track 1 of 2: 100%|██████████| 11/11 [00:11<00:00,  1.05s/it]

Processing speakers: 100%|██████████| 8/8 [06:02<00:00, 45.37s/it]


Failed: []


## 16 – Hinweis: Keine Evaluation auf dem Eval-Set

Das Eval-Set hat keine Labels – `evaluate.py` kann nicht ausgeführt werden.
Die generierten VTT-Dateien (`output_EVAL_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20`)
werden direkt als Einreichung verwendet.
Die endgültigen WER-Metriken auf dem Eval-Set werden erst nach der Einreichung
durch den CHiME-Organisatoren bekannt gegeben.
